<h1>
Computational Intelligence Assignment 
</h1>
<p>
Implement SVM and KNN without any library. 
</p>

In [ ]:
# upload data
from google.colab import files
uploaded = files.upload()

Saving adult.csv to adult.csv


First of all, I've used `google.colab` to uploud my data and read them by `pandas`

In [ ]:
import io
import pandas as pd
import numpy as np
import sklearn.model_selection
features = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
              "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
              "Hours per week", "Country", "Target"] 
df = pd.read_csv(io.StringIO(uploaded['adult.csv'].decode('utf-8')), names=features, sep=r'\s*,\s*', engine='python')
df.head()

,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K



<h3>
The data must be preprocessed before using them for training 
</h3>


---


<p>
<ul>
<li>Missing data</li>
<p>
If missing data of every column is more than 10% so I'll remove the column otherwise I'll remove the records</p>

In [ ]:
def preprocess_missing_data(data):
  for c in data.columns:
    number_of_missing_data = data[c].isin(["?"]).sum()
    if number_of_missing_data > 0:
      percentage = float(number_of_missing_data)/data.shape[0]*100
      print("percentage of records with missig data: ", percentage,"%")
      if percentage < 10:
        data = data[data[c]!="?"]
      else :
        data = data.drop(c, axis=1)
  return data


df = preprocess_missing_data(df)
df.head()

percentage of records with missig data:  5.7307235575938735 %
percentage of records with missig data:  0.021718828052038314 %
percentage of records with missig data:  1.7617795929007454 %


,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


<ul>
<li>
Normalize Numerical Features
</li>

In [ ]:
def normalize_numerical_features(data):
  data_nums = data.select_dtypes(include=np.number)
  for c in data_nums.columns:
        max = data[c].max()
        min = data[c].min()
        data[c] = (data[c]-min)/(max-min)
  return data

df = normalize_numerical_features(df)
df.head()

,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,0.301370,State-gov,0.043350,Bachelors,0.800000,Never-married,Adm-clerical,Not-in-family,White,Male,0.02174,0.0,0.397959,United-States,<=50K
1,0.452055,Self-emp-not-inc,0.047274,Bachelors,0.800000,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00000,0.0,0.122449,United-States,<=50K
2,0.287671,Private,0.136877,HS-grad,0.533333,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.00000,0.0,0.397959,United-States,<=50K
3,0.493151,Private,0.149792,11th,0.400000,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.00000,0.0,0.397959,United-States,<=50K
4,0.150685,Private,0.219998,Bachelors,0.800000,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.00000,0.0,0.397959,Cuba,<=50K


<ul>
<li>One-Hot Encoding</li>
<p>for preprocess non-numerical feature except the Target!</p>
</ul>

In [ ]:
def one_hot_encoding(data):
  # Y:
  data['Target'] = data['Target'].replace('<=50K', 0).replace('>50K', 1)
  # X:
  data_cat = data.select_dtypes(exclude=np.number)
  data = pd.get_dummies(data, columns=data_cat.columns)
  return data

df = one_hot_encoding(df)
df.tail()

,Age,fnlwgt,Education-Num,Capital Gain,Capital Loss,Hours per week,Target,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Private,Workclass_Self-emp-inc,Workclass_Self-emp-not-inc,Workclass_State-gov,Workclass_Without-pay,Education_10th,Education_11th,Education_12th,Education_1st-4th,Education_5th-6th,Education_7th-8th,Education_9th,Education_Assoc-acdm,Education_Assoc-voc,Education_Bachelors,Education_Doctorate,Education_HS-grad,Education_Masters,Education_Preschool,Education_Prof-school,Education_Some-college,Martial Status_Divorced,Martial Status_Married-AF-spouse,Martial Status_Married-civ-spouse,Martial Status_Married-spouse-absent,Martial Status_Never-married,Martial Status_Separated,Martial Status_Widowed,Occupation_Adm-clerical,Occupation_Armed-Forces,Occupation_Craft-repair,...,Country_Canada,Country_China,Country_Columbia,Country_Cuba,Country_Dominican-Republic,Country_Ecuador,Country_El-Salvador,Country_England,Country_France,Country_Germany,Country_Greece,Country_Guatemala,Country_Haiti,Country_Holand-Netherlands,Country_Honduras,Country_Hong,Country_Hungary,Country_India,Country_Iran,Country_Ireland,Country_Italy,Country_Jamaica,Country_Japan,Country_Laos,Country_Mexico,Country_Nicaragua,Country_Outlying-US(Guam-USVI-etc),Country_Peru,Country_Philippines,Country_Poland,Country_Portugal,Country_Puerto-Rico,Country_Scotland,Country_South,Country_Taiwan,Country_Thailand,Country_Trinadad&Tobago,Country_United-States,Country_Vietnam,Country_Yugoslavia
48836,0.219178,0.156895,0.8,0.000000,0.0,0.397959,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48837,0.301370,0.136723,0.8,0.000000,0.0,0.357143,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48839,0.287671,0.244762,0.8,0.000000,0.0,0.500000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48840,0.369863,0.047666,0.8,0.054551,0.0,0.397959,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48841,0.246575,0.114195,0.8,0.000000,0.0,0.602041,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0



<p>Divide train, validation and test set from x and y</p>

In [ ]:
X = df.drop("Target", axis=1)
Y = np.array(df["Target"])
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(X, Y, test_size=0.2)
x_train.head()
x_train,x_validation,y_train,y_validation=sklearn.model_selection.train_test_split(x_train, y_train, test_size=0.25)

**SVM**


---



In [ ]:
from sklearn.svm import SVC

def SVM_prediction(x_train, y_train, x_test):
  svclassifier = SVC(kernel='linear')
  svclassifier.fit(x_train, y_train)
  y_pred = svclassifier.predict(x_test)
  return y_pred


y_pred = SVM_prediction(x_train, y_train, x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
print("accuracy : ", accuracy, "%")

accuracy :  0.8442233278054173 %



**KNN**


---





In [ ]:
def find_destinations(x, x_train):
  destinations = []
  # for i in range(1, len(x_train)):
  #   dest = np.sqrt(np.sum(np.power(np.array(x) - np.array(x_train.iloc[i]), 2)))
  #   destinations.append(dest)
  dest = np.sqrt(np.sum(np.power(np.array(x) - np.array(x_train), 2), axis=1))
  return dest
  
destinations = find_destinations(x_test.iloc[1], x_train)
destinations

array([3.53643188, 3.48069672, 3.17342478, ..., 2.85973997, 2.85555558,
       3.47154462])

In [ ]:
def predicate_y(y_train, indexes):
  sum = 0
  for i in indexes:
    sum += y_train[i]
  possibility = sum/len(indexes)
  return possibility

In [ ]:
def KNN(x_train, y_train, x_validation, y_validation, K=5):
  y_pre = []
  for i in range(0, len(x_validation)):
    dests = find_destinations(x_validation.iloc[i], x_train)
    dests = np.array(dests)
    indexes = dests.argsort()[:K]
    pos = predicate_y(y_train, indexes)
    pre = 0 if pos < 0.5 else 1
    if i % 2000 == 0 :
      print("predicate : ", pre , "  actual : ", y_validation[i])
    y_pre.append(pre)
  return y_pre


Find the best K for the algorithm


---



In [ ]:
max_accuracy = 0
best_K = 0;
for k in range(5, 30, 5):
  print("K=", k)
  y_pre = KNN(x_train, y_train, x_validation, y_validation, k)
  accuracy = np.sum(y_pre == y_validation) / len(y_pre) * 100
  print("Accuracy: %.2F%% " %accuracy)
  print("")
  if accuracy > max_accuracy:
    max_accuracy = accuracy
    best_K = k

y_pre = KNN(x_train, y_train, x_test, y_test, best_K) 
accuracy = np.sum(y_pre == y_test) / len(y_pre) * 100
print("Accuracy with K = ", best_K, " : %.2F%% " %accuracy)

K= 5
predicate :  0   actual :  0
predicate :  1   actual :  0
predicate :  1   actual :  1
predicate :  0   actual :  0
predicate :  0   actual :  1
Accuracy: 82.29% 

K= 10
predicate :  0   actual :  0
predicate :  0   actual :  0
predicate :  1   actual :  1
predicate :  0   actual :  0
predicate :  0   actual :  1
Accuracy: 82.51% 

K= 15
predicate :  0   actual :  0
predicate :  0   actual :  0
predicate :  1   actual :  1
predicate :  0   actual :  0
predicate :  0   actual :  1
Accuracy: 83.20% 

K= 20
predicate :  0   actual :  0
predicate :  0   actual :  0
predicate :  1   actual :  1
predicate :  0   actual :  0
predicate :  0   actual :  1
Accuracy: 83.29% 

K= 25
predicate :  0   actual :  0
predicate :  0   actual :  0
predicate :  1   actual :  1
predicate :  0   actual :  0
predicate :  0   actual :  1
Accuracy: 83.48% 

predicate :  0   actual :  0
predicate :  0   actual :  0
predicate :  0   actual :  0
predicate :  0   actual :  0
predicate :  1   actual :  1
Accura